In [6]:
#########################
# Problem Set 3
# MACS 30100, Dr. Evans
# Yinxian Zhang
#########################

import numpy as np
import scipy as sp
import numpy.linalg as lin
import scipy.integrate as intgr
import scipy.stats as sts
import scipy.optimize as opt
import matplotlib.pyplot as plt
import pandas as pd
import os


# load income data
inc = np.loadtxt('incomes.txt')
# len(inc)


def trunc_lognorm_pdf(xvals, mu, sigma, cutoff):
    '''
    --------------------------------------------------------------------
    Generate pdf values for the lognormal distribution with mean mu and standard deviation sigma. If the cutoff is given, then the PDF values are inflated upward to reflect the zero probability on values above the
    cutoff. 
    --------------------------------------------------------------------
    INPUTS:
    xvals  = (N,) vector, values of the log normally distributed random
             variable
    mu     = scalar, mean of the log normally distributed random variable
    sigma  = scalar > 0, standard deviation of the log normally distributed
             random variable
    cutoff = scalar or string, ='None' if no cutoff is given, otherwise
             is scalar upper bound value of distribution. Values above
             this value have zero probability
    
    OTHER FUNCTIONS AND FILES CALLED BY THIS FUNCTION: None
    
    OBJECTS CREATED WITHIN FUNCTION:
    prob_notcut = scalar 
    pdf_vals = (N,) vector, lognormal PDF values for mu and sigma
               corresponding to xvals data
    
    FILES CREATED BY THIS FUNCTION: None
    
    RETURNS: pdf_vals
    --------------------------------------------------------------------
    '''
    if cutoff == 'None':
        prob_notcut = 1.0 
    else:
        prob_notcut = sts.lognorm.cdf(cutoff, s=sigma, loc=mu, scale=np.exp(mu)) # use lognormal CDF function
            
    pdf_vals = ((1/(xvals * sigma * np.sqrt(2 * np.pi)) *
                    np.exp( - (np.log(xvals) - mu)**2 / (2 * sigma**2))) / prob_notcut) # log normal distribution function 
    
    return pdf_vals 


def data_moments(xvals): ## compute mean and standard deviation statistics of the real data
    '''
    --------------------------------------------------------------------
    This function computes the two data moments for GMM
    (mean(data), variance(data)).
    --------------------------------------------------------------------
    INPUTS:
    xvals = (N,) vector, test scores data
    
    OTHER FUNCTIONS AND FILES CALLED BY THIS FUNCTION: None
    
    OBJECTS CREATED WITHIN FUNCTION:
    mean_data = scalar, mean value of test scores data
    std_data  = scalar > 0, standard deviation of test scores data
    
    FILES CREATED BY THIS FUNCTION: None
    
    RETURNS: mean_data, std_data
    --------------------------------------------------------------------
    '''
    mean_data = xvals.mean()
    std_data = xvals.std()
    
    return mean_data, std_data


def model_moments(mu, sigma, cutoff): # compute mean and standard deviation statistics of model 
    '''
    --------------------------------------------------------------------
    This function computes the two model moments for GMM
    (mean(model data), variance(model data)).
    --------------------------------------------------------------------
    INPUTS:
    mu     = scalar, mean of the normally distributed random variable
    sigma  = scalar > 0, standard deviation of the normally distributed
             random variable
    cutoff = scalar or string, ='None' if no cutoff is given, otherwise
             is scalar upper bound value of distribution. Values above
             this value have zero probability
    
    OTHER FUNCTIONS AND FILES CALLED BY THIS FUNCTION:
        trunc_norm_pdf()
        xfx()
        x2fx()
    
    OBJECTS CREATED WITHIN FUNCTION:
    mean_model = scalar, mean value of test scores from model
    m_m_err    = scalar > 0, estimated error in the computation of the
                 integral for the mean of the distribution
    std_model  = scalar > 0, standard deviation of test scores from model
    v_m_err    = scalar > 0, estimated error in the computation of the
                 integral for the variance of the distribution
    
    FILES CREATED BY THIS FUNCTION: None
    
    RETURNS: mean_model, std_model
    --------------------------------------------------------------------
    '''
    xfx = lambda x: x * trunc_lognorm_pdf(x, mu, sigma, cutoff)
    (mean_model, m_m_err) = intgr.quad(xfx, 0.000001, cutoff) # xvals should not be zero in lognormal PDF function
    x2fx = lambda x: ((x - mean_model) ** 2) * trunc_lognorm_pdf(x, mu, sigma, cutoff) 
    (var_model, v_m_err) = intgr.quad(x2fx, 0.000001, cutoff)
    std_model = np.sqrt(var_model)
    
    return mean_model, std_model


def err_vec(xvals, mu, sigma, cutoff, simple): # call data and model statistics to compute error
    '''
    --------------------------------------------------------------------
    This function computes the vector of moment errors (in percent
    deviation from the data moment vector) for GMM.
    --------------------------------------------------------------------
    INPUTS:
    xvals  = (N,) vector, test scores data
    mu     = scalar, mean of the normally distributed random variable
    sigma  = scalar > 0, standard deviation of the normally distributed
             random variable
    cutoff = scalar or string, ='None' if no cutoff is given, otherwise
             is scalar upper bound value of distribution. Values above
             this value have zero probability
    simple = boolean, =True if errors are simple difference, =False if
             errors are percent deviation from data moments
    
    OTHER FUNCTIONS AND FILES CALLED BY THIS FUNCTION:
        data_moments()
        model_moments()
    
    OBJECTS CREATED WITHIN FUNCTION:
    mean_data  = scalar, mean value of data
    var_data   = scalar > 0, variance of data
    moms_data  = (2, 1) matrix, column vector of two data moments
    mean_model = scalar, mean value from model
    var_model  = scalar > 0, variance from model
    moms_model = (2, 1) matrix, column vector of two model moments
    err_vec    = (2, 1) matrix, column vector of two moment error
                 functions
    
    FILES CREATED BY THIS FUNCTION: None
    
    RETURNS: err_vec
    --------------------------------------------------------------------
    '''
    mean_data, std_data = data_moments(xvals)
    moms_data = np.array([[mean_data], [std_data]]) # data moments
    mean_model, std_model = model_moments(mu, sigma, cutoff)
    moms_model = np.array([[mean_model], [std_model]]) # model moments
    if simple: 
        err_vec = moms_model - moms_data # simple difference between data and model moments
    else:
        err_vec = (moms_model - moms_data) / moms_data # percent deviation between data and model moments
    
    return err_vec

def criterion(params, *args): # compute the criterion function
    '''
    --------------------------------------------------------------------
    This function computes the GMM weighted sum of squared moment errors
    criterion function value given parameter values and an estimate of
    the weighting matrix.
    --------------------------------------------------------------------
    INPUTS:
    params = (2,) vector, ([mu, sigma])
    mu     = scalar, mean of the normally distributed random variable
    sigma  = scalar > 0, standard deviation of the normally distributed
             random variable
    args   = length 3 tuple, (xvals, cutoff, W_hat)
    xvals  = (N,) vector, values of the truncated normally distributed
             random variable
    cutoff = scalar or string, ='None' if no cutoff is given, otherwise
             is scalar upper bound value of distribution. Values above
             this value have zero probability
    W_hat  = (R, R) matrix, estimate of optimal weighting matrix
    
    OTHER FUNCTIONS AND FILES CALLED BY THIS FUNCTION:
        norm_pdf()
    
    OBJECTS CREATED WITHIN FUNCTION:
    err        = (2, 1) matrix, column vector of two moment error
                 functions
    crit_val   = scalar > 0, GMM criterion function value
    
    FILES CREATED BY THIS FUNCTION: None
    
    RETURNS: crit_val
    --------------------------------------------------------------------
    '''
    mu, sigma = params
    xvals, cutoff, W = args
    err = err_vec(xvals, mu, sigma, cutoff, simple=False) # get error variance-covariance matrix
    crit_val = np.dot(np.dot(err.T, W), err)  # plug in weighting matrix
    
    return crit_val

# estimate the GMM parameters with a simple Indentity matrix
mu_init = 11
sig_init = 0.3
params_init = np.array([mu_init, sig_init]) # set up inital guess of parameters

W_hat = np.eye(2) # two by two identity matrix
gmm_args = (inc, 150000.0, W_hat) # plug in data and weighting matrix
results = opt.minimize(criterion, params_init, args=(gmm_args), method='L-BFGS-B', bounds=((1e-10, None), (1e-10, None))) # the divided number should not be zero
mu_GMM1, sig_GMM1 = results.x

# get criterion function values at the estimated GMM parameter values
params_GMM = np.array([mu_GMM1, sig_GMM1])
GMM_Value = criterion(params_GMM, *gmm_args)[0][0]

# compare model moments and data moments
mean_data, std_data = data_moments(inc)
mean_model, std_model = model_moments(mu_GMM1, sig_GMM1, 150000)
# Q1(c): two-step GMM estimator 
#------------------------------

# step 1: use vector of moment error function from part b to get a new estimate of the inverse variance-covariance matrix
err1 = err_vec(inc, mu_GMM1, sig_GMM1, 150000, False) # plug in GMM estimated parameters to get error vector 
VCV2 = np.dot(err1, err1.T) / inc.shape[0] # calculate the sum of squared errors 
W_hat2 = lin.pinv(VCV2)  # Use the pseudo-inverse calculated by SVD to get the two-step optimal weighting matrix 

#step 2: re-estimate the GMM estimator using the optimal two-step weighting matrix.
params_init_2 = np.array([mu_GMM1, sig_GMM1]) 
gmm_args2 = (inc, 150000.0, W_hat2) # plug in the optimal weighting matrix
results2 = opt.minimize(criterion, params_init_2, args=(gmm_args2), method='L-BFGS-B', bounds=((1e-10, None), (1e-10, None))) # run once again the minimizer
mu_GMM2, sig_GMM2 = results2.x # get the two-step estimated parameters

# get criterion function values at the estimated GMM parameter values
params_GMM2 = np.array([mu_GMM2, sig_GMM2])
GMM2_Value = criterion(params_GMM2, *gmm_args2)[0][0]

# compare model moments and data moments
mean_data, std_data = data_moments(inc)
mean_model2, std_model2 = model_moments(mu_GMM2, sig_GMM2, 150000)

In [7]:
plt.hist(inc, bins = 30, normed = True, label = 'Actual Outcomes')
plt.title("Expected Incomes of MACSS Students 2018-2020")
plt.xlabel("Incomes")
plt.ylabel("Percentage of MACSS Graduates")
plt.plot(inc_sim, trunc_lognorm_pdf(inc_sim, mu_GMM1, sig_GMM1, 150000), color = 'g', label = 'GMM1: $\mu$=11.33,$\sigma$=0.21')
plt.plot(inc_sim, trunc_lognorm_pdf(inc_sim, mu_GMM2, sig_GMM2, 150000), color = 'b', label = 'GMM2: $\mu$=12.40,$\sigma$=1.36')
plt.legend(loc = 'upper left')

NameError: name 'inc_sim' is not defined